In [ ]:
import pandas as pd
import os
import torch
from torch.utils.data import DataLoader
import albumentations as A

from datasets.lungdatasets import SchenzenMontgomeryLungSegmentationDataset
from datasets.lungdatasets import CheXpertLungSegmentationDataset
from models.unet import ResNetUNet
from utils.utils import bce_dice_loss, dice_metric
import numpy as np
from tqdm import tqdm
import glob
import cv2

CHEXPERT_TRAIN = '../CheXpert-v1.0-small/train.csv'
BASE_MASKS = './intermediate/out_lung_mask3/'
BASE_IMG = './data/chexpert-cardio-nofinding/'
BASE_EXTRA = 'CheXpert-v1.0-small/train/'

def get_transforms(size, test = True):
    #TODO: Do test-time augmentation?
    if test:
        return A.Compose([
        A.Resize(height=size, width=size, p=1.0)
        ])
    return A.Compose([
        A.Resize(height=size, width=size, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.3),
        A.Transpose(p=0.3),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.2, rotate_limit=45, p=0.3),
    ])

#Find Min skipping 0
def find_min(arr):
    min_val = 1000
    for idx, value in enumerate(arr):
        if value < min_val and value != 0:
            min_val = value
    return min_val

def find_chest_width_image(img,post_process=True):
    if post_process:
        img = post_process_image(img)
    start = np.argmax(img[:,:,1],axis=1)
    end = np.argmax(img[:,::-1,1],axis=1)
    h,w,c = img.shape
    return find_min(start), w - find_min(end), w

def find_chest_width(path,post_process=True):
    img = cv2.imread(path)
    if post_process:
        img = post_process_image(img)
    start = np.argmax(img[:,:,1],axis=1)
    end = np.argmax(img[:,::-1,1],axis=1)
    h,w,c = img.shape
    return find_min(start), w - find_min(end), w

def post_process_image(img,hull = True):
    
    dst = img[:,:,0]
    
    #kernel = np.ones((3, 3), np.uint8)
    #dst = cv2.erode(dst, kernel,iterations= 3) 

    contours, hierarchy = cv2.findContours(dst, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #create an empty image for contours
    img_contours = np.zeros(img.shape)
    # draw the contours on the empty image
    cs = [(c,cv2.contourArea(c)) for c in contours]
    cs.sort(key=lambda x:x[1])
    if hull:
        hulls = [cv2.convexHull(p[0]) for p in cs[-2:]]
        cv2.drawContours(img_contours, hulls, -1, (0,255,0), -1)
    else:
        contours2 = [p[0] for p in cs[-2:]]
        cv2.drawContours(img_contours, contours2, -1, (0,255,0), -1)
    return img_contours

def find_img(path):
    img = cv2.imread(path)
    return img.shape

import numpy as np
import cv2
from matplotlib.pyplot import imshow

ds_train_no_finding = CheXpertLungSegmentationDataset("./data/hand-label/nofinding.json", '../CheXpert-v1.0-small/train/'
                                                      , aug_transform=get_transforms(320)
                                                     , test = True)
ds_train_cardiomegaly = CheXpertLungSegmentationDataset("./data/hand-label/cardiomegaly-certain.json", '../CheXpert-v1.0-small/train/'
                                                        , aug_transform=get_transforms(320)
                                                       , test = True)

full_ds_chexpert1 = torch.utils.data.ConcatDataset([ds_train_no_finding, ds_train_cardiomegaly])
train_ds_chexpert1,val_ds_chexpert1 = torch.utils.data.random_split(full_ds_chexpert1, [len(full_ds_chexpert1) - 100, 100], generator=torch.Generator().manual_seed(42))
sample = np.uint8(full_ds_chexpert1[0][1].cpu().numpy() * 255)

In [ ]:
ignore = [x[2] for x in val_ds_chexpert1]

In [ ]:
#Find Min skipping 0
def find_min(arr):
    min_val = 1000
    for idx, value in enumerate(arr):
        if value < min_val and value != 0:
            min_val = value
    return min_val

def find_chest_width(path,post_process=True):
    img = cv2.imread(path)
    if post_process:
        img = post_process_image(img)
    start = np.argmax(img[:,:,1],axis=1)
    end = np.argmax(img[:,::-1,1],axis=1)
    h,w,c = img.shape
    return find_min(start), w - find_min(end), w

def post_process_image(img,hull = True):
    
    dst = img[:,:,0]
    
    #kernel = np.ones((3, 3), np.uint8)
    #dst = cv2.erode(dst, kernel,iterations= 3) 

    contours, hierarchy = cv2.findContours(dst, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #create an empty image for contours
    img_contours = np.zeros(img.shape)
    # draw the contours on the empty image
    cs = [(c,cv2.contourArea(c)) for c in contours]
    cs.sort(key=lambda x:x[1])
    if hull:
        hulls = [cv2.convexHull(p[0]) for p in cs[-2:]]
        cv2.drawContours(img_contours, hulls, -1, (0,255,0), -1)
    else:
        contours2 = [p[0] for p in cs[-2:]]
        cv2.drawContours(img_contours, contours2, -1, (0,255,0), -1)
    return img_contours

def find_img(path):
    img = cv2.imread(path)
    return img.shape

In [ ]:
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

img = cv2.imread(BASE_MASKS + 'patient00005_study1_view1_frontal.jpg')

plt.imshow(cv2.cvtColor(img[:,:,0], cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Image after Component Labeling")
plt.show()

plt.imshow(post_process_image(img))
plt.axis('off')
plt.title("Image after Component Labeling")
plt.show()

In [ ]:
find_chest_width(BASE_MASKS + 'patient00005_study1_view1_frontal.jpg',post_process=True)

In [ ]:
find_chest_width(BASE_MASKS + 'patient00005_study1_view1_frontal.jpg',post_process=False)

In [ ]:
(img[:,:,0] > 0)[100]

In [ ]:
import pandas as pd

df = pd.read_csv(CHEXPERT_TRAIN)
df = df.fillna(0)
df.head()

In [ ]:
import pickle
import cv2


with open('./intermediate/out_heartmodel2AFTERPRETRAINING4.pickle', 'rb') as handle:
     out_heart = pickle.load(handle)
        
with open('./intermediate/out_pathsmodel2AFTERPRETRAINING4.pickle', 'rb') as handle:
     out_paths = pickle.load(handle)

In [ ]:
import numpy as np
from tqdm import tqdm

def calculate_ctr(top_k=5):
    ctr_count_no = 0
    ctr_amount_no = 0

    ctr_count_un = 0
    ctr_amount_un = 0

    ctr_count = 0
    ctr_amount = 0

    ctr_healthy = []
    ctr_uncertain = []
    ctr_cardiomegaly = []
    
    processed = []

    for path, heart in tqdm(list(zip(out_paths,out_heart))):
        #if path in ignore:
        #    continue
        boxes = heart[1]['boxes']
        #print(path)
        boxes_to_consider = boxes[0:top_k]
       
        widths_in_pixels = (boxes_to_consider[:,2] - boxes_to_consider[:,0]).cpu().numpy() 
        min_val,max_val,width = find_chest_width(BASE_MASKS + path)

        h,w,c = find_img(BASE_IMG + path)
        heart = widths_in_pixels.sum()/(top_k*w)
        chest = (max_val-min_val)/width

        row = df[df.Path.eq(BASE_EXTRA + path.replace('_','/',2))]
        #print('-------------------')
        #print(abs(row[row.columns[5:]].sum()).sum())
        #print(row['Sex'].values[0])
        #print(row['Support Devices'].values[0])
        #print(row['Cardiomegaly'])
        #print(row['No Finding'].values)
        clazz = -1
        uncertain = False
        if row['No Finding'].values[0] == 1:
            ctr_count_no += 1
            ctr_amount_no += heart/chest
            ctr_healthy.append(heart/chest)
            clazz = 0
        elif row['Cardiomegaly'].values[0] == 1:
            ctr_count += 1
            ctr_amount += heart/chest
            ctr_cardiomegaly.append(heart/chest)
            clazz = 1
        else:
            ctr_count_un += 1
            ctr_amount_un += heart/chest
            ctr_uncertain.append(heart/chest)
            uncertain = True
            clazz = 1
            
        ctr =  heart/chest
        processed.append(
            (path,ctr,clazz,heart,chest,boxes_to_consider,uncertain,
             row['Sex'].values[0],row['Support Devices'].values[0],abs(row[row.columns[5:]].sum()).sum())
        )
    return processed,ctr_amount_no,ctr_count_no,ctr_amount_un,ctr_count_un,ctr_amount,ctr_count,ctr_healthy,ctr_cardiomegaly,ctr_uncertain
    

In [ ]:
p, ctr_amount_no,ctr_count_no,ctr_amount_un,ctr_count_un,ctr_amount,ctr_count,ctr_healthy,ctr_cardiomegaly,ctr_uncertain = calculate_ctr(top_k=1)

In [ ]:
optimal_threshold = 0
optimal_accuracy = 0

for i in range(100):
    threshold = i/100
    c = 0
    for example in p:
        prediction = -1
        #if example[2] == -1:
        #    continue
        if example[1] >= threshold:
            prediction = 1
        else:
            prediction = 0
        correct = prediction == example[2]
        if correct:
            c += 1

    if c/(len(p)) > optimal_accuracy:
        optimal_accuracy = c/(len(p))
        optimal_threshold = threshold
        
print(optimal_threshold, optimal_accuracy)

In [ ]:
ctr_amount_no/ctr_count_no,ctr_amount_un/ctr_count_un,ctr_amount/ctr_count

In [ ]:
print(len(ctr_uncertain))
print(len([x for x in ctr_uncertain if x<.55 and x > .45]))
print(len([x for x in ctr_cardiomegaly if x<.55 and x > .45]))
print(len([x for x in ctr_healthy if x<.55 and x > .45]))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

distributionA = np.random.normal(0.3, 0.1, 1000)
distributionB = np.random.normal(0.5, 0.1, 1000)
distributionC = np.random.normal(0.7, 0.1, 1000)

labels = np.array(range(20))/20
no_finding = np.histogram(ctr_healthy,bins=np.arange(21)/20)
uncertain = np.histogram(ctr_uncertain,bins=np.arange(21)/20)
cardiomegaly = np.histogram(ctr_cardiomegaly,bins=np.arange(21)/20)

x = np.arange(len(labels)) *2 # the label locations
width = 0.5# the width of the bars



fig, ax = plt.subplots(figsize=(10,5))

plt.axvline(x=np.average(ctr_healthy)*40, color='blue', linestyle='--')
plt.axvline(x=np.average(ctr_uncertain)*40, color='orange', linestyle='--')
plt.axvline(x=np.average(ctr_cardiomegaly)*40, color='green', linestyle='--')

rects1 = ax.bar(x - width, no_finding[0], width, label='No Finding')
rects2 = ax.bar(x, uncertain[0], width, label='Uncertain')
rects3 = ax.bar(x + width, cardiomegaly[0] , width, label='Cardiomegaly')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Count')
ax.set_xlabel('Predicted cardiothoracic ratio')
ax.set_title('Predicted cardiothoracic ratio count grouped by ground-truth label')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
fig.tight_layout()

plt.show()


In [ ]:
p[0]

In [ ]:
ctr_count_no,ctr_count_un,ctr_count

In [ ]:
wrong_uncertain = [x for x in p if x[1] < 0.50  and x[2] == 1 and x[6] is True]
wrong_certain = [x for x in p if x[1] < 0.50  and x[2] == 1 and x[6] is False]
len(wrong_certain), len(wrong_uncertain)
wrong = [x for x in p if x[1] > 0.55  and x[2] == 0]

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)

img = cv2.imread(BASE_MASKS + wrong[idx][0])
path = '../CheXpert-v1.0-small/train/' + wrong[idx][0].replace('_','/',2)
img2 = cv2.imread(path)

h,w,c = find_img(BASE_IMG + wrong[idx][0])
print(wrong[idx])
x0,y0,x1,y1 = wrong[idx][5].cpu().numpy()[0]
#x0,y0,x1,y1
print('Box ', x0,y0,x1,y1)
img3 = cv2.rectangle(img2, (x0,y0), (x1,y1), (0, 0, 255), 2)

f, axarr = plt.subplots(1,3)
axarr[0].imshow(img[:,:,0])
axarr[1].imshow(post_process_image(img,hull=False))
axarr[2].imshow(img3)

print(df[df.Path == 'CheXpert-v1.0-small/train/' + wrong[idx][0].replace('_','/',2)])
print(idx)
idx +=1


In [ ]:
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from matplotlib.pyplot import figure

def show_image(image_path,bounding_box):
    plt.rcParams["figure.figsize"] = (20,10)

    img = cv2.imread(BASE_MASKS + image_path)
    path = '../CheXpert-v1.0-small/train/' + image_path.replace('_','/',2)
    img2 = cv2.imread(path)

    h,w,c = find_img(BASE_IMG + image_path)
    x0,y0,x1,y1 = bounding_box
    print('Box ', x0,y0,x1,y1)
    img3 = cv2.rectangle(img2, (x0,y0), (x1,y1), (0, 0, 255), 2)

    f, axarr = plt.subplots(1,3)
    axarr[0].imshow(img[:,:,0])
    axarr[1].imshow(post_process_image(img,hull=False))
    axarr[2].imshow(img3)

    print(df[df.Path == 'CheXpert-v1.0-small/train/' + image_path.replace('_','/',2)])



In [ ]:
p[0]

In [ ]:
[x for x in p if x[8] != 1 and x[8] != 0]

In [ ]:
np.unique(np.array([x[9] for x in p]))

In [ ]:
correct =  np.array([x[2] for x in p if x[9] == 1])
predicted =  np.array([x[1] for x in p  if x[9] == 1]

In [ ]:
correct2 =  np.array([x[2] for x in p if x[9] == 2])
predicted2 =  np.array([x[1] for x in p  if x[9] == 2])

In [ ]:
correct3 = np.array([x[2] for x in p if x[9] == 3])
predicted3 = np.array([x[1] for x in p  if x[9] == 3])

In [ ]:
correct4 = np.array([x[2] for x in p if x[9] == 4])
predicted4 = np.array([x[1] for x in p  if x[9] == 4])

In [ ]:
correct5 = np.array([x[2] for x in p if x[9] > 5])
predicted5 = np.array([x[1] for x in p  if x[9] > 5])

In [ ]:
(correct == (predicted > 0.5)).mean()

In [ ]:
(correct2 == (predicted2 > 0.5)).mean()

In [ ]:
(correct3 == (predicted3 > 0.5)).mean()

In [ ]:
(correct4 == (predicted4 > 0.5)).mean()

In [ ]:
(correct5 == (predicted5 > 0.5)).mean()

In [ ]:
len(correct5)

In [ ]:
len(correct), len(correct2), len(correct3), len(correct4), len(correct5)

In [ ]:
sample = [x for x in p if x[0] == 'patient14064_study1_view1_frontal.jpg'][0]
show_image(sample[0],sample[5].cpu().numpy()[0])

wrong_uncertain = [x for x in p if x[1] < 0.50  and x[2] == 1 and x[6] is True]
wrong_certain = [x for x in p if x[1] < 0.50  and x[2] == 1 and x[6] is False]
len(wrong_certain), len(wrong_uncertain)
wrong = [x for x in p if x[1] > 0.55  and x[2] == 0]

In [ ]:
study = [x for x in p if x[1] <0.45 and x[2] == 1]
idx = 0
len(study)

In [ ]:
np.array([x[1] for x in study]).mean()

In [ ]:
show_image(study[idx][0],study[idx][5].cpu().numpy()[0])
print(study[idx][1])
print(idx)
idx+=1

In [ ]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot


ns_probs = [0 for _ in range(len(correct))]
ns_auc = roc_auc_score(correct, ns_probs)
lr_auc = roc_auc_score(correct, predicted)
lr_auc2 = roc_auc_score(correct2, predicted2)
#lr_auc3 = roc_auc_score(correct3, predicted3)
#lr_auc4 = roc_auc_score(correct4, predicted4)
#lr_auc5 = roc_auc_score(correct5, predicted5)
print('No Skill: ROC AUC=%.7f' % (ns_auc))
print('Logistic: ROC AUC=%.7f' % (lr_auc))
print('Logistic: ROC AUC=%.7f' % (lr_auc2))
#print('Logistic: ROC AUC=%.7f' % (lr_auc3))
#print('Logistic: ROC AUC=%.7f' % (lr_auc4))
#print('Logistic: ROC AUC=%.7f' % (lr_auc5))

ns_fpr, ns_tpr, _ = roc_curve(correct, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(correct,  predicted)
lr_fpr2, lr_tpr2, _ = roc_curve(correct2,  predicted2)
#lr_fpr3, lr_tpr3, _ = roc_curve(correct3,  predicted3)
#lr_fpr4, lr_tpr4, _ = roc_curve(correct4,  predicted4)
#lr_fpr5, lr_tpr5, _ = roc_curve(correct5,  predicted5)
plt.rcParams["figure.figsize"] = (10,5)
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='Baseline')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Model finetuned - 1 observation')
pyplot.plot(lr_fpr2, lr_tpr2, marker='.', label='Model finetuned - 2 observations')
#pyplot.plot(lr_fpr3, lr_tpr3, marker='.', label='Model finetuned - Uncertain about support devices')
#pyplot.plot(lr_fpr4, lr_tpr4, marker='.', label='Model finetuned - No Support devices')
#pyplot.plot(lr_fpr5, lr_tpr5, marker='.', label='Model finetuned - Uncertain about support devices')
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')

pyplot.legend()

pyplot.show()
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(correct, (np.array(predicted) > 0.50)*1, average='macro')